# Deep Convolutional GANs

[구글 Colab에서 실행하기](https://colab.research.google.com/github/aonekoda/DL-PyTorch/blob/master/04VAE_GAN/04DCGAN_Exercise.ipynb)

이번 실습에서는 generator 와 discriminator에 convolutional layers를 사용하는 GAN 모형을 생성한다. 이것을 DCGAN 이라고 한다. 


DCGAN 을 [Street View House Numbers](http://ufldl.stanford.edu/housenumbers/) (SVHN) dataset에 훈련하여 집 주소 이미지를 생성한다. 앞 서 실습한 MNIST보다는 생성해야 할 이미지가 복잡하다. 

![img](../assets/svhn_dcgan.png)

DCGAN 을 사용하여 그럴듯한 images 를 생성한다. 
* Load in and pre-process the house numbers dataset
* Define discriminator and generator networks
* Train these adversarial networks
* Visualize the loss over time and some sample, generated images

#### Deeper Convolutional Networks

SVHN은 MNIST data에 비해 복잡하다. 따라서 여러개의 convolutional 과 transpose convolutional layers 를 사용하여  discriminator와 generator를 구현한다. batch normalization기법도 사용된다. 


In [ ]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

%matplotlib inline

## Getting the data

SVHN dataset을 다운로드한다. Tensor datatypes으로 적절히 transform 하고 적절한 batch size를 가지는 dataloaders 로 생성한다.

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms

# Tensor transform
transform = transforms.ToTensor()

# SVHN training datasets
svhn_train = datasets.SVHN(root='data/', split='train', download=True, transform=transform)

batch_size = 128
num_workers = 0

# build DataLoaders for SVHN dataset
train_loader = torch.utils.data.DataLoader(dataset=svhn_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_workers)


### Visualize the Data

각각의 이미지는 32x32 사이즈의 3 color channels (RGB)이미지이다. 각각의 이미지에는 숫자 label이 붙어있다.

In [ ]:
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
plot_size=20
for idx in np.arange(plot_size):
    ax = fig.add_subplot(2, plot_size/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.transpose(images[idx], (1, 2, 0)))
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

### Pre-processing: scaling from -1 to 1

pre-processing; DCGAN은 `tanh` activated generator를 사용하므로 픽셀 값을 -1 to 1로 rescale한다. 처음 읽어들이면 0-1 사이 값으로 되어 있다.

In [ ]:
# current range
img = images[0]

print('Min: ', img.min())
print('Max: ', img.max())

In [ ]:
# helper scale function
def scale(x, feature_range=(-1, 1)):
    ''' Scale takes in an image x and returns that image, scaled
       with a feature_range of pixel values from -1 to 1. 
       This function assumes that the input x is already scaled from 0-1.'''
    # assume x is scaled to (0, 1)
    # scale to feature_range and return scaled x
    # TODO 
    
    return x


In [ ]:
# scaled range
scaled_img = scale(img)

print('Scaled min: ', scaled_img.min())
print('Scaled max: ', scaled_img.max())

---
# Define the Model



## Discriminator

maxpooling layers를 사용하지 않는다. 
* discriminator 의 input은 32x32x3 tensor images
* convolutional, hidden layers를 사용한다.
* output은 fully connected layer이다; sigmoid output를 사용하지 않고 loss function으로 [BCEWithLogitsLoss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss)를 사용한다.


![img](../assets/conv_discriminator.png)

convolutional layers 의 depth를 32 -> 64 ->128...로 늘려나간다. maxpooling layers을 사용하지 않고 stride를 사용하여 down-sample을 수행한다.

batch normalization 을 사용한다. [nn.BatchNorm2d](https://pytorch.org/docs/stable/nn.html#batchnorm2d) **다만 first convolutional layer 와 final에서는 사용하지 않는다**. 

#### Helper `conv` function 

convolution > batch norm > leaky ReLU를 반복적으로 사용하여 layer를 구성하므로 별도의 helper 함수를 만들어 간단하게 사용한다. convolutional + optional batch norm layer로 이루어진 sequential series로 모형을 구성한다 . 


**kernel_size = 4** , **stride = 2** 를 사용한다.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# helper conv function
def conv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels, out_channels, 
                           kernel_size, stride, padding, bias=False)
    
    # append conv layer
    layers.append(conv_layer)

    if batch_norm:
        # append batchnorm layer
        layers.append(nn.BatchNorm2d(out_channels))
     
    # using Sequential container
    return nn.Sequential(*layers)


In [ ]:
class Discriminator(nn.Module):

    def __init__(self, conv_dim=32):
        super(Discriminator, self).__init__()

        # complete init function
        
        #TODO 
        
    def forward(self, x):
        # complete forward function
        
        
        #TODO 
        
        
        return x
    

## Generator

G의 input 은 noise vector `z`이다. ouput은 우리가 생성해야할 32X32X3의 이미지이다.

![img](../assets/conv_generator.png)


transpose convolutional layers 를 사용하여 images를 생성한다. 

* 첫 layer는 fully connected layer이다. 4x4x512 같은 w,h가 작고 depth가 큰 형태로 시작한다. 
* batch normalization과 leaky ReLU activation를 사용한다. 
* [transpose convolutional layers](https://pytorch.org/docs/stable/nn.html#convtranspose2d)을 사용하여 w, h를 더블로 늘려나가고 depth는 줄여서 원하는 이미지 사이즈로 생성한다.
* batch normalization 과 ReLU 를 모든 hidden layers에 사용하는데 마지막은  `tanh` activation을 사용한다.

#### Helper `deconv` function

transpose convolution > batch norm > ReLU의 반복으로 D를 구성한다. 이를 손쉽게 하기 위해 helper함수를 하나 만든다.


Note: transpose convolutions에서 **kernel_size = 4** and a **stride = 2**로 설정한다 .

In [ ]:
# helper deconv function
def deconv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    """Creates a transposed-convolutional layer, with optional batch normalization.
    """
    ## TODO: Complete this function
    ## create a sequence of transpose + optional batch norm layers

    return nn.Sequential(*layers)


In [ ]:
class Generator(nn.Module):
    
    def __init__(self, z_size, conv_dim=32):
        super(Generator, self).__init__()

        # complete init function
        # TODO
        
    def forward(self, x):
        # complete forward function
        #TODO 
        
        return x
    

## Build complete network



In [ ]:
# define hyperparams
conv_dim = 32
z_size = 100

# define discriminator and generator
D = Discriminator(conv_dim)
G = Generator(z_size=z_size, conv_dim=conv_dim)

print(D)
print()
print(G)

### Training on GPU



In [ ]:
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    # move models to GPU
    G.cuda()
    D.cuda()
    print('GPU available for training. Models moved to GPU')
else:
    print('Training on CPU.')
    

---
## Discriminator and Generator Losses

앞 선 예제에서와 같이 D와 G의 loss함수를 정한다.

### Discriminator Losses

> * discriminator에서 total loss 는  real 와 fake images의 합이다. `d_loss = d_real_loss + d_fake_loss`. 
* discriminator는 real images에 대해서 output 1, fake images에 대해서는 0 을 출력한다.

* `D(real_images) = 1`. 
* `D(fake_images) = 0`, 
* `fake_images = G(z)`. 

### Generator Loss

G는`D(fake_images) = 1`이 되어야 한다. 즉 generates (fakes)가 real로 판별되도록 해야 한다.

In [ ]:
def real_loss(D_out, smooth=False):
    batch_size = D_out.size(0)
    # label smoothing
    if smooth:
        # smooth, real labels = 0.9
        labels = torch.ones(batch_size)*0.9
    else:
        labels = torch.ones(batch_size) # real labels = 1
    # move labels to GPU if available     
    if train_on_gpu:
        labels = labels.cuda()
    # binary cross entropy with logits loss
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

def fake_loss(D_out):
    batch_size = D_out.size(0)
    labels = torch.zeros(batch_size) # fake labels = 0
    if train_on_gpu:
        labels = labels.cuda()
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

## Optimizers

연구에 따르면 DCGAN model 에서는 learning_rate를 작게 설정한다.

### Hyperparameters

GANs은 hyperparameters 설정에 민감하다. 이에 대해서는 [DCGAN paper](https://arxiv.org/pdf/1511.06434.pdf) 을 참조한다.

In [ ]:
import torch.optim as optim

# params
# TODO

lr = 
beta1=  # momentum decay
beta2=  # scale decay

# Create optimizers for the discriminator and generator
d_optimizer = optim.Adam(D.parameters(), lr, [beta1, beta2])
g_optimizer = optim.Adam(G.parameters(), lr, [beta1, beta2])

---
## Training

discriminator 와 the generator를 번갈아 가며 훈련한다.  `real_loss` 와 `fake_loss` 를 사용하여 훈련한다.

### Discriminator training
1. real(훈련 데이터)에 대해서 discriminator loss 를 계산        
2. fake images를 생성
3. fake(generated image)에 대한discriminator loss 를 구한다.     
4. total lpss = real logg + fake loss
5. backpropagation + optimization을 수행하여 discriminator의 weights를 훈련한다.

### Generator training
1. fake images를 생성한다.
2. fake에 대한 discriminator loss 를 계산한다. D(fake_image) = 1
3. backpropagation + optimization step 을 수행하여 generator의 weights를 훈련한다.

#### Saving Samples

loss statistics 와 generated "fake" samples을 저장한다.

**Evaluation mode**

generator 가 sample 이미지를 생성할 때는 evaluation mode를 사용한다: `G.eval()`. evaluation 모드에서는 batch normalization layers가 training에 사용한 평균과 분산을 사용하지 않는다. 그리고 dropout layers도 off된다.

In [ ]:
import pickle as pkl

# training hyperparams
num_epochs = 30

# keep track of loss and generated, "fake" samples
samples = []
losses = []

print_every = 300

# Get some fixed data for sampling. These are images that are held
# constant throughout training, and allow us to inspect the model's performance
sample_size=16
fixed_z = np.random.uniform(-1, 1, size=(sample_size, z_size))
fixed_z = torch.from_numpy(fixed_z).float()

# train the network
for epoch in range(num_epochs):
    
    for batch_i, (real_images, _) in enumerate(train_loader):
                
        batch_size = real_images.size(0)
        
        # important rescaling step
        real_images = scale(real_images)
        
        # ============================================
        #            TRAIN THE DISCRIMINATOR
        # ============================================
        
        d_optimizer.zero_grad()
        
        # 1. Train with real images

        # Compute the discriminator losses on real images 
        if train_on_gpu:
            real_images = real_images.cuda()
        
        D_real = D(real_images)
        d_real_loss = real_loss(D_real)
        
        # 2. Train with fake images
        
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(batch_size, z_size))
        z = torch.from_numpy(z).float()
        # move x to GPU, if available
        if train_on_gpu:
            z = z.cuda()
        fake_images = G(z)
        
        # Compute the discriminator losses on fake images            
        D_fake = D(fake_images)
        d_fake_loss = fake_loss(D_fake)
        
        # add up loss and perform backprop
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        
        # =========================================
        #            TRAIN THE GENERATOR
        # =========================================
        g_optimizer.zero_grad()
        
        # 1. Train with fake images and flipped labels
        
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(batch_size, z_size))
        z = torch.from_numpy(z).float()
        if train_on_gpu:
            z = z.cuda()
        fake_images = G(z)
        
        # Compute the discriminator losses on fake images 
        # using flipped labels!
        D_fake = D(fake_images)
        g_loss = real_loss(D_fake) # use real loss to flip labels
        
        # perform backprop
        g_loss.backward()
        g_optimizer.step()

        # Print some loss stats
        if batch_i % print_every == 0:
            # append discriminator loss and generator loss
            losses.append((d_loss.item(), g_loss.item()))
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, d_loss.item(), g_loss.item()))

    
    ## AFTER EACH EPOCH##    
    # generate and save sample, fake images
    G.eval() # for generating samples
    if train_on_gpu:
        fixed_z = fixed_z.cuda()
    samples_z = G(fixed_z)
    samples.append(samples_z)
    G.train() # back to training mode


# Save training generator samples
with open('train_samples.pkl', 'wb') as f:
    pkl.dump(samples, f)

## Training loss



In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()

## Generator samples from training

generator가 생성한 이미지를 화면에 출력한다.

In [ ]:
# helper function for viewing a list of passed in sample images
def view_samples(epoch, samples):
    fig, axes = plt.subplots(figsize=(16,4), nrows=2, ncols=8, sharey=True, sharex=True)
    for ax, img in zip(axes.flatten(), samples[epoch]):
        img = img.detach().cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        img = ((img +1)*255 / (2)).astype(np.uint8) # rescale to pixel range (0-255)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        im = ax.imshow(img.reshape((32,32,3)))

In [ ]:
_ = view_samples(-1, samples)